# YOLO via ultralytics + CycleGAN dataset Augmentation

Utilizar el siguiente cuaderno para aplicar CycleGAN en Data Augmentatio

### Setting GAN tool

In [1]:
!python --version

Python 3.10.12


In [ ]:
import os
import shutil

In [2]:
# Solo ejecutar si se tiene targeta de video AMD  

!export PYTORCH_ROCM_ARCH="gfx1031"
!export HIP_VISIBLE_DEVICES=0
!export ROCM_PATH=/opt/rocm

## Copia de imagenes del dataset y descargo del modelo de cycleGAN

In [3]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Clonando en 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2516 (delta 0), reused 1 (delta 0), pack-reused 2513
Recibiendo objetos: 100% (2516/2516), 8.20 MiB | 15.50 MiB/s, listo.
Resolviendo deltas: 100% (1575/1575), listo.


In [14]:
!pip install -r pytorch-CycleGAN-and-pix2pix/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 KB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 KB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 KB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 KB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 40.9 MB/s eta 0:00:00
Using legacy 'setup.py install' for visdom, since package 'wheel' is not installed.
  Running setup.py install for visdom ... done


In [ ]:
!cp -r ../datasets/v6/train/images/ ./images
!cp -r ../datasets/v6/train/labels// ./labels

In [10]:
!sh pytorch-CycleGAN-and-pix2pix/scripts/download_cyclegan_model.sh summer2winter_yosemite


Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [summer2winter_yosemite]
AVISO: el «timestamping» no hace nada cuando se combina con
la opción -O. Consulte el manual para más detalles.

--2024-06-18 22:35:05--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/summer2winter_yosemite.pth
Resolviendo efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Conectando con efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)[128.32.244.190]:80... 

conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 45575747 (43M)
Guardando como: ‘./checkpoints/summer2winter_yosemite_pretrained/latest_net_G.pth’

./checkpoints/summe 100%[===================>]  43,46M   384KB/s    en 1m 57s  

2024-06-18 22:37:03 (380 KB/s) - ‘./checkpoints/summer2winter_yosemite_pretrained/latest_net_G.pth’ guardado [45575747/45575747]



In [23]:
!sh pytorch-CycleGAN-and-pix2pix/scripts/download_cyclegan_model.sh winter2summer_yosemite

Note: available models are apple2orange, orange2apple, summer2winter_yosemite, winter2summer_yosemite, horse2zebra, zebra2horse, monet2photo, style_monet, style_cezanne, style_ukiyoe, style_vangogh, sat2map, map2sat, cityscapes_photo2label, cityscapes_label2photo, facades_photo2label, facades_label2photo, iphone2dslr_flower
Specified [winter2summer_yosemite]
AVISO: el «timestamping» no hace nada cuando se combina con
la opción -O. Consulte el manual para más detalles.

--2024-06-18 23:04:46--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/winter2summer_yosemite.pth
Resolviendo efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Conectando con efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)[128.32.244.190]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 45575747 (43M)
Guardando como: ‘./checkpoints/winter2summer_yosemite_pretrained/latest_net_G.pth’

./checkpoints/winte 100%[===================>]  43,46M  

In [ ]:
# eliminar_imagenes_repetidas
# la cycleGAN utiliza test para generar imágenes fake pero copia imágenes reales.
# Se eliminan las reales y nos quedamos con las fake generadas por CycleGAN.
# Se deja el mismo nombre pero con _fake al final.
def eliminar_imagenes_repetidas(image_directory):
  i = 0
  for filename in os.listdir(image_directory):
      if filename.endswith((".jpg", ".jpeg", ".png", ".gif")) and "fake" not in filename:
          file_path = os.path.join(image_directory, filename)
          os.remove(file_path)
          i +=1

  print(f"{i} imágenes sin 'fake' eliminadas")

# imprimir_cantidad_de_archivos
# imprime la cantidad de archivos en un directorio
def imprimir_cantidad_de_archivos(directorio,descripcion):
    v = os.listdir(directorio)
    print(descripcion + ': ' + str(len(v)))

# copiar_imagenes
# copia imágenes de un directorio fuente a un directorio destino
def copiar_imagenes(fuente,destino):
  image_extensions = (".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff")
  i = 0
  for filename in os.listdir(fuente):
      if filename.lower().endswith(image_extensions):
          source_path = os.path.join(fuente, filename)
          destination_path = os.path.join(destino, filename)
          shutil.copy(source_path, destination_path)
      i +=1
  print(f"{i} imágenes han sido copiadas.")

# print_fake
# imprime la cantidad de imágenes falsas generadas por CycleGAN
def print_fake(directorio,descripcion):
  v = os.listdir(directorio)
  i = 0
  for image in v:
    if "fake" in image:
      i +=1
  print(descripcion + ': ' + str(i))

# duplicar_etiquetas_fake
# para el par de directorios directorio_imagenes,directorio_etiquetas,
# duplica los labels de aquellas imágenes generadas mediante cycleGAN
def duplicar_etiquetas_fake(directorio_imagenes, directorio_etiquetas):
  for filename in os.listdir(directorio_imagenes):
      if "fake" in filename:
          base_name = filename.split("_fake")[0]
          label_filename = base_name + ".txt"
          original_label_path = os.path.join(directorio_etiquetas, label_filename)

          if os.path.exists(original_label_path):
              new_label_filename = base_name + "_fake.txt"
              new_label_path = os.path.join(directorio_etiquetas, new_label_filename)
              shutil.copy(original_label_path, new_label_path)


### Train 2612 imágenes GAN, demora alrededor de 2 imágenes por segundo

In [21]:
cant_img = len(os.listdir('images'))

In [22]:
!python pytorch-CycleGAN-and-pix2pix/test.py --dataroot images \
--name summer2winter_yosemite_pretrained \
--display_winsize 640 \
--crop_size 640 \
--load_size 640 \
--model test \
--num_test {cant_img} \
--no_dropout \
--results_dir resultados/train

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 640                           	[default: 256]
                 dataroot: images                        	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 640                           	[default: 256]
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter:

In [24]:

!python pytorch-CycleGAN-and-pix2pix/test.py --dataroot images \
--name winter2summer_yosemite_pretrained \
--display_winsize 640 \
--crop_size 640 \
--load_size 640 \
--model test \
--num_test {cant_img} \
--no_dropout \
--results_dir resultados/train

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 640                           	[default: 256]
                 dataroot: images                        	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 640                           	[default: 256]
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter:

#### Eliminamos todas las imagenes no generadas

In [43]:
base_dir_resultados_gan = 'resultados/train/'


In [46]:
# Eliminamos las imagenes que no sean fake

win_dir = 'winter2summer_yosemite_pretrained/test_latest/images/'
sum_dir = 'summer2winter_yosemite_pretrained/test_latest/images/'
imagenesGeneradosWinNoFiltrado = os.listdir(base_dir_resultados_gan + win_dir)
imagenesGeneradosSumNoFiltrado = os.listdir(base_dir_resultados_gan + sum_dir)

for path in imagenesGeneradosWinNoFiltrado:
    if '_real' in path:
        os.remove(base_dir_resultados_gan + win_dir + path)
    else :
        partes_nombres = path.split('.png')
        os.rename(base_dir_resultados_gan + win_dir + path, base_dir_resultados_gan + win_dir + partes_nombres[0] + '_win' + '.png')

for path in imagenesGeneradosSumNoFiltrado:
    if '_real' in path:
        os.remove(base_dir_resultados_gan + sum_dir + path)
    else :
        partes_nombres = path.split('.png')
        os.rename(base_dir_resultados_gan + sum_dir + path, base_dir_resultados_gan + sum_dir + partes_nombres[0] + '_sum' + '.png')        

In [48]:
imagenesGeneradosWin = os.listdir(base_dir_resultados_gan + win_dir)
imagenesGeneradosSum = os.listdir(base_dir_resultados_gan + sum_dir)

for path in imagenesGeneradosWin:
    os.rename(base_dir_resultados_gan + win_dir + path, base_dir_resultados_gan + 'resultados_gans/' + path)

for path in imagenesGeneradosSum:
    os.rename(base_dir_resultados_gan + sum_dir + path, base_dir_resultados_gan + 'resultados_gans/' + path)

In [56]:
pathGans = os.listdir(base_dir_resultados_gan + 'resultados_gans/' + 'images/')

for path_gans in pathGans:
    real_path = path_gans.split('_fake')[0]
    real_path = real_path.split('resultados_gans')[1] + '.txt'

    shutil.copy('labels/' + real_path, base_dir_resultados_gan + 'resultados_gans/labels/' + path_gans + '.txt')


Luego se deben copiar los labels e imagenes en el data sets